In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
import pandas as pd

In [3]:
df=pd.read_csv("train_clean_data(1).csv",index_col=0)
df.head()

,Loan_ID,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,gender_Male,married_Yes,education_Not Graduate,property_area_Semiurban,property_area_Urban,self_employed_Yes,Loan_status_Y
1,LP001003,1,4583,1508.0,128.0,360.0,1.0,1,1,0,0,0,0,0
2,LP001005,0,3000,0.0,66.0,360.0,1.0,1,1,0,0,1,1,1
3,LP001006,0,2583,2358.0,120.0,360.0,1.0,1,1,1,0,1,0,1
4,LP001008,0,6000,0.0,141.0,360.0,1.0,1,0,0,0,1,0,1
5,LP001011,2,5417,4196.0,267.0,360.0,1.0,1,1,0,0,1,1,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 517 entries, 1 to 613
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Loan_ID                  517 non-null    object 
 1   Dependents               517 non-null    object 
 2   ApplicantIncome          517 non-null    int64  
 3   CoapplicantIncome        517 non-null    float64
 4   LoanAmount               517 non-null    float64
 5   Loan_Amount_Term         517 non-null    float64
 6   Credit_History           517 non-null    float64
 7   gender_Male              517 non-null    int64  
 8   married_Yes              517 non-null    int64  
 9   education_Not Graduate   517 non-null    int64  
 10  property_area_Semiurban  517 non-null    int64  
 11  property_area_Urban      517 non-null    int64  
 12  self_employed_Yes        517 non-null    int64  
 13  Loan_status_Y            517 non-null    int64  
dtypes: float64(4), int64(8), o

In [5]:
    df.shape

(517, 14)

In [6]:
df['Loan_status_Y'].value_counts()

1    360
0    157
Name: Loan_status_Y, dtype: int64

In [7]:
'''Train Test Split'''
X=df.drop(['Loan_ID','Dependents','Loan_status_Y'],axis=1)
y=df['Loan_status_Y']

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train,y_test=train_test_split(X,y,test_size=0.33,stratify=y,random_state=42)

In [10]:
!pip install --upgrade scikit-learn
!pip install --upgrade imbalanced-learn

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [11]:
from imblearn.over_sampling import SMOTE
'''set the random state, it ensures that the synthetic data generated during oversampling process is the same evert time you run the code'''
smote=SMOTE(random_state=42)
X_train_res, y_train_res=smote.fit_resample(X_train, y_train)

In [12]:
model1=DecisionTreeClassifier(class_weight='balanced')
model2=GaussianNB()
model3=LogisticRegression(solver='liblinear',max_iter=200,class_weight='balanced')

In [13]:
model1.fit(X_train_res,y_train_res)
model2.fit(X_train_res,y_train_res)
model3.fit(X_train_res,y_train_res)

LogisticRegression(class_weight='balanced', max_iter=200, solver='liblinear')

In [14]:
'''averaging probabilities is called as soft voting or probabilistic ensembling '''
pred1=model1.predict_proba(X_test)
pred2=model2.predict_proba(X_test)
pred3=model3.predict_proba(X_test)

In [15]:
final_pred=(pred1+pred2+pred3)/3

In [16]:
final_pred

array([[0.3922371 , 0.6077629 ],
       [0.40659709, 0.59340291],
       [0.02848188, 0.97151812],
       [0.04511306, 0.95488694],
       [0.35097038, 0.64902962],
       [0.23817237, 0.76182763],
       [0.29479916, 0.70520084],
       [0.10438884, 0.89561116],
       [0.47694479, 0.52305521],
       [0.09476928, 0.90523072],
       [0.13907491, 0.86092509],
       [0.14761545, 0.85238455],
       [0.06334219, 0.93665781],
       [0.11690039, 0.88309961],
       [0.10704855, 0.89295145],
       [0.52204014, 0.47795986],
       [0.05678684, 0.94321316],
       [0.047642  , 0.952358  ],
       [0.97920343, 0.02079657],
       [0.0778268 , 0.9221732 ],
       [0.77405917, 0.22594083],
       [0.59550255, 0.40449745],
       [0.03753766, 0.96246234],
       [0.07021892, 0.92978108],
       [0.62788638, 0.37211362],
       [0.40903575, 0.59096425],
       [0.02358324, 0.97641676],
       [0.16701356, 0.83298644],
       [0.09439592, 0.90560408],
       [0.61470582, 0.38529418],
       [0.

In [17]:
model1.classes_

array([0, 1], dtype=int64)

In [19]:
clf1=DecisionTreeClassifier(class_weight='balanced')
clf2=GaussianNB()
clf3=LogisticRegression(solver='liblinear',max_iter=200,class_weight='balanced')

In [20]:
eclf1=VotingClassifier(
    estimators=[('DT',clf1),('GNB',clf2),('LR',clf3)],
    voting='hard')
eclf1=eclf1.fit(X_train_res,y_train_res)

In [21]:
predictions=eclf1.predict(X_test)

In [22]:
predictions

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0], dtype=int64)

In [23]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

              precision    recall  f1-score   support

           0       0.70      0.62      0.65        52
           1       0.84      0.88      0.86       119

    accuracy                           0.80       171
   macro avg       0.77      0.75      0.76       171
weighted avg       0.80      0.80      0.80       171

[[ 32  20]
 [ 14 105]]


In [26]:
eclf2=VotingClassifier(
    estimators=[('DT',clf1),('GNB',clf2),('LR',clf3)],
    voting='soft')
eclf2=eclf1.fit(X_train_res,y_train_res)

In [28]:
predictions=eclf2.predict(X_test)

In [29]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

              precision    recall  f1-score   support

           0       0.71      0.62      0.66        52
           1       0.84      0.89      0.87       119

    accuracy                           0.81       171
   macro avg       0.78      0.75      0.76       171
weighted avg       0.80      0.81      0.80       171

[[ 32  20]
 [ 13 106]]


In [30]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
base_model1=DecisionTreeClassifier()
bagging=BaggingClassifier(
    estimator=base_model1, n_estimators=200, random_state=0)
bagging.fit(X_train_res, y_train_res)
y_pred_bagging=bagging.predict(X_test)
print("Bagging Accuracy:",accuracy_score(y_test, y_pred_bagging))

Bagging Accuracy: 0.7660818713450293
